In [ ]:
import zipfile
import os

zip_path = "/content/soil-classification-part-2.zip"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("soil-classification")

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img
import joblib

# Config
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
df = pd.read_csv('/content/soil-classification/soil_competition-2025/test_ids.csv')
clf, scaler = joblib.load('soil_ocsvm_model.pkl')

# Feature extraction
def extract_features(path):
    img = load_img(path, target_size=(224, 224))
    arr = np.expand_dims(img, axis=0)  # Add batch dimension
    return model.predict(arr).flatten()

# Prediction loop
df['predicted_label'] = [
    (1 if clf.predict(scaler.transform(extract_features(Path(test_dir)/row['image_id']).reshape(1,-1)))[0] == 1 else 0)
    if Path.exists(Path(test_dir)/row['image_id']) else -1
    for _, row in df.iterrows()
]

df.to_csv('test_predictions.csv', index=False)
print(f"✅ Saved {len(df)} predictions")
